In [23]:
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import datetime
import re
import cx_Oracle
import numpy as np
import datetime

# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

username = 'ebs_apps'
password = 'ebs_apps'
host = 'csdwddb.cswg.com' #EBSD2
port = '1521'
service_name = 'DWD_USR' #EBSD2

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
oracle_con = cx_Oracle.connect(username, password, dsn)
oracle_cur = oracle_con.cursor()

post_cur.execute("""SELECT *
FROM XXPO_PULLBAK_PARAM
WHERE LPAD(CAST(to_fac_code AS VARCHAR), 3, '0') IN (
    SELECT LPAD(CAST(facility_code AS VARCHAR), 3, '0')
    FROM cspoms_replenishment_config
    WHERE POMS_SOURCE = 'CSPOMS'
    AND STATUS = 'Y'
)""")
data = post_cur.fetchall()

# Get column names from the cursor description
column_names = [desc[0] for desc in post_cur.description]
print(column_names)
df = pd.DataFrame(data, columns=column_names)
df = df[['from_fac_code','from_warehouse_num','to_fac_code',
         'incr','vendor_number','max_pallets_allowed',
         'max_weight_allowed','max_cube_allowed','lead_time','op_days',
        'creation_date','created_by']]

# Dictionary with new column names
new_column_names = {'from_fac_code': "T2_TABLE_26_FR_FAC",
                    'from_warehouse_num': "T2_TABLE_26_FR_WH",
                    'to_fac_code': "T2_TABLE_26_TO_FAC",
                    'incr': "T2_TABLE_26_ORD_INCR",
                    'vendor_number': "T2_TABLE_26_VENDOR",
                    'max_pallets_allowed': "T2_TABLE_26_MIN_PALLET",
                    'max_weight_allowed': "T2_TABLE_26_MIN_WGHT",
                    'max_cube_allowed': "T2_TABLE_26_MIN_CUBE",
                    'lead_time': "T2_TABLE_26_LEAD_TIME",
                    'op_days': "T2_TABLE_26_OP_DAYS"}
df.rename(columns=new_column_names, inplace=True)
print(df)


# Get the current system date and time
current_date = datetime.datetime.now()

# Format the date according to your requirements
formatted_date = current_date.strftime("%Y-%m-%d %H:%M:%S")  # Adjust the format as needed

# Assign the formatted date to the DataFrame column
df['last_update_date'] = formatted_date
df['request_id'] = -999
df['process_status'] = "N"

# Display the updated DataFrame
df

table_name = "P2P_DATA.XXPO_PULLBACK_BICEPS2EXA_STG"
fields = [col for col in df.columns]
str_sub = ', '.join(['%s'] * len(fields))
# Construct the INSERT query with column names
insert_query = f"INSERT INTO {table_name} ({', '.join(fields)}) VALUES ({', '.join([':' + str(i+1) for i in range(len(fields))])})"

# Execute the INSERT statement with the data
oracle_cur.executemany(insert_query, data_tuples)

# Execute the INSERT statement with the data
oracle_cur.executemany(insert_query, data_tuples)
count = oracle_cur.rowcount
oracle_con.commit()
print(f'INSERTED {count} EXCEL RECORDS TO {table_name} TABLE')


['seq_num', 'from_warehouse_num', 'from_fac_code', 'from_fac_name', 'to_fac_code', 'to_fac_name', 'incr', 'vendor_number', 'max_pallets_allowed', 'max_weight_allowed', 'max_cube_allowed', 'lead_time', 'op_days', 'creation_date', 'created_by', 'last_updated_date', 'last_updated_by', 'request_id']
  T2_TABLE_26_FR_FAC T2_TABLE_26_FR_WH T2_TABLE_26_TO_FAC  \
0                 15                 2                 23   
1                 15                 2                 43   

  T2_TABLE_26_ORD_INCR T2_TABLE_26_VENDOR T2_TABLE_26_MIN_PALLET  \
0                    P             777965                     60   
1                    P             777965                     60   

  T2_TABLE_26_MIN_WGHT T2_TABLE_26_MIN_CUBE T2_TABLE_26_LEAD_TIME  \
0                42000                 1500                     1   
1                42000                 1500                     1   

  T2_TABLE_26_OP_DAYS creation_date created_by  
0                   3    2024-01-01          1  
1       

DatabaseError: ORA-01861: literal does not match format string

In [31]:
print(df)
print(df.info())


  T2_TABLE_26_FR_FAC T2_TABLE_26_FR_WH T2_TABLE_26_TO_FAC  \
0                 15                 2                 23   
1                 15                 2                 43   

  T2_TABLE_26_ORD_INCR T2_TABLE_26_VENDOR T2_TABLE_26_MIN_PALLET  \
0                    P             777965                     60   
1                    P             777965                     60   

  T2_TABLE_26_MIN_WGHT T2_TABLE_26_MIN_CUBE T2_TABLE_26_LEAD_TIME  \
0                42000                 1500                     1   
1                42000                 1500                     1   

  T2_TABLE_26_OP_DAYS creation_date created_by    last_update_date  \
0                   3    2024-01-01          1 2024-03-27 13:36:35   
1                   3    2024-01-01          1 2024-03-27 13:36:35   

   request_id process_status     last_updated_date  
0        -999              N  27-Mar-2024 13:38:02  
1        -999              N  27-Mar-2024 13:38:02  
<class 'pandas.core.frame.DataFra

In [30]:
# Convert date columns to datetime format
df['creation_date'] = pd.to_datetime(df['creation_date'])
df['last_update_date'] = pd.to_datetime(df['last_update_date'])
# Convert numeric columns to integer format
df['request_id'] = df['request_id'].astype(int)

In [37]:
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
import datetime
import cx_Oracle

# Database connection parameters
# Replace these with your actual database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

username = 'ebs_apps'
password = 'ebs_apps'
host = 'csdwddb.cswg.com' #EBSD2
port = '1521'
service_name = 'DWD_USR' #EBSD2

# Create a connection to Oracle
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
oracle_con = cx_Oracle.connect(username, password, dsn)
oracle_cur = oracle_con.cursor()

# Retrieve data from PostgreSQL
post_cur.execute("""SELECT *
FROM XXPO_PULLBAK_PARAM
WHERE LPAD(CAST(to_fac_code AS VARCHAR), 3, '0') IN (
    SELECT LPAD(CAST(facility_code AS VARCHAR), 3, '0')
    FROM cspoms_replenishment_config
    WHERE POMS_SOURCE = 'CSPOMS'
    AND STATUS = 'Y'
)""")
data = post_cur.fetchall()

# Get column names from the cursor description
column_names = [desc[0] for desc in post_cur.description]

# Create a DataFrame from the fetched data
df = pd.DataFrame(data, columns=column_names)

# Select the required columns and rename them
df = df[['from_fac_code', 'from_warehouse_num', 'to_fac_code',
         'incr', 'vendor_number', 'max_pallets_allowed',
         'max_weight_allowed', 'max_cube_allowed', 'lead_time', 'op_days',
         'creation_date', 'created_by']]

# Rename the columns according to the desired format
new_column_names = {'from_fac_code': "T2_TABLE_26_FR_FAC",
                    'from_warehouse_num': "T2_TABLE_26_FR_WH",
                    'to_fac_code': "T2_TABLE_26_TO_FAC",
                    'incr': "T2_TABLE_26_ORD_INCR",
                    'vendor_number': "T2_TABLE_26_VENDOR",
                    'max_pallets_allowed': "T2_TABLE_26_MIN_PALLET",
                    'max_weight_allowed': "T2_TABLE_26_MIN_WGHT",
                    'max_cube_allowed': "T2_TABLE_26_MIN_CUBE",
                    'lead_time': "T2_TABLE_26_LEAD_TIME",
                    'op_days': "T2_TABLE_26_OP_DAYS"}
df.rename(columns=new_column_names, inplace=True)

# Convert date columns to datetime format
df['creation_date'] = pd.to_datetime(df['creation_date'])

# Get the current system date and time
current_date = datetime.datetime.now().date()

# Assign the current date to the last_update_date column
df['last_update_date'] = current_date

# Set the request_id, process_status, and last_updated_date columns
df['request_id'] = -999
df['process_status'] = "N"

# Insert the DataFrame into Oracle
table_name = "P2P_DATA.XXPO_PULLBACK_BICEPS2EXA_STG"
fields = [col for col in df.columns]
str_sub = ', '.join([':%d' % (i + 1) for i in range(len(fields))])
insert_query = f"INSERT INTO {table_name} ({', '.join(fields)}) VALUES ({str_sub})"
oracle_cur.executemany(insert_query, df.values.tolist())
insert_count = oracle_cur.rowcount
oracle_con.commit()
# merge_query = f"""
# select * from XXPO_PULLBACK_BICEPS2EXA_DATA where RECORD_ID IS NULL
# """
# print(merge_query)
# oracle_cur.execute(merge_query)
# count = oracle_cur.rowcount
# print(f'MERGED {count} EXCEL RECORDS TO {table_main} TABLE')

oracle_con.commit()
# Print a confirmation message
print(f'Inserted {insert_count} records into {table_name} table.')


Inserted 2 records into P2P_DATA.XXPO_PULLBACK_BICEPS2EXA_STG table.
